In [123]:
pip install PyPDF2

In [124]:
pip  install openai

In [125]:
pip install -U sentence-transformers


In [126]:
pip install chromadb

In [127]:
import openai
##import langchain
from PyPDF2 import PdfReader
##from langchain.text_splitter import RecursiveCharacterTextSplitter
##import  as db
from openai import OpenAI
import os
import pandas as pd

In [131]:
import chromadb
client = chromadb.Client()

In [132]:
def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return text

In [133]:
def read_pdfs_in_directory(directory):
    pdf_texts = []
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):
            file_path = os.path.join(directory, filename)
            ##pdf_texts[filename] = read_pdf(file_path)
            content = read_pdf(file_path)
            pdf_texts.append({"file_name": filename, "content": content})
    return pdf_texts

In [134]:
directory_path = "/content/sample_data/documents"
pdf_texts = read_pdfs_in_directory(directory_path)
#df = pd.DataFrame(list(pdf_texts.items()), columns=['Name', 'Content'])

In [135]:
pdf_texts

[{'file_name': 'CharlotteMayResume.pdf',
  'content': 'CHARLOTTE MAY\nSenior Data Scientist | CAP | DASCA\n+1-713-645-3682 latishaham@hey.com www.linkedin.com/in/--lhamilton--\nPhoenix, AZ\nSUMMARY\nWith eight years experience working as a data scientist and more than ten  \nyears of strong academic background in analytical modeling and  \nprogramming, I strive to bring high-quality business intelligence solutions  \nto businesses seeking higher eﬃciency. With expertise in advanced  \nstatistical algorithms, machine learning and forecasting, I have also worked  \non over 1150 high performance parallel processing applications using C++  \nand C.\nEXPERIENCE\nSenior Data Scientist - Marketing Team\nDell\n2018 - Present \xa0 Phoenix, AZ\nTurned data into actionable insights, providing C-suite stakeholders with  \ninsightful recommendations to streamline business operations and  \nimprove customer experience.\nGenerated statistical reports and visualizations, providing key insights  \nfor 

In [136]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L3-v2')

In [137]:
documents = []
embeddings = []
metadatas = []
ids = []
for index,data in enumerate(pdf_texts):
    documents.append(data['content'])
    embeding = model.encode(data['content']).tolist()
    embeddings.append(embeding)
    metadatas.append({'source':data['file_name']})
    ids.append(str(index+1))

In [139]:
resume_collection = client.create_collection("resumepdf_collection")
resume_collection.add(
    documents=documents,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids
)

print(resume_collection)

name='resumepdf_collection' id=UUID('615ee3a5-8318-4012-89f9-e414cfa49732') metadata=None tenant='default_tenant' database='default_database'


In [140]:
query = "Which candidates belong to Data Science?"
input_em = model.encode(query).tolist()

results = resume_collection.query(
    query_embeddings=[input_em],
    n_results=2
)
results

{'ids': [['2', '1']],
 'distances': [[22.187475204467773, 24.048749923706055]],
 'metadatas': [[{'source': 'JennieLinnResume.pdf'},
   {'source': 'CharlotteMayResume.pdf'}]],
 'embeddings': None,
 'documents': [["Innovative Data Scientist Intern\n+1-541-754-3010 help@enhancv.com linkedin.com\nNew York, NYJENNIE \nLINN\nSUMMARY\nRising data science professional with a strong interest in the insurance industry. \nProven ability to build predictive models and analytic solutions. Proficient in R, \nPython, and SQL.\nEXPERIENCE\nData Scientist Intern 2023 \x00 2023\nArch Capital Services LLC Jersey City, NJ\nCollaborated with experienced modelers to build predictive models and analytic \nsolutions. Researched and tested new analytics approaches and tools. Assisted in \ntesting and implementation of models.\nCollaborated with experienced modelers to build predictive models, \ncontributing to a 10% increase in efficiency\nResearched and tested new analytics approaches, leading to a 15% \nimpr